<a href="https://colab.research.google.com/github/jeromeslash83/Data-Science/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill

In [24]:
# Read data into a Pandas DataFrame and combine header rows
data = pd.read_excel('PROD_WSTVC001_C077P006_Data Extraction_en_20230330124607.xlsx', header=[0, 1], sheet_name=1)
data.columns = [' '.join(col).strip() for col in data.columns.values]

/usr/local/lib/python3.9/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [25]:
data.columns

Index(['Project No. Protocol No.', 'Site No. Site No.',
       'Subject Id Screening No.', 'Visit Visit', 'Form Sequence Seq No.',
       'DM.BRTHDAT Date of Birth', 'DM.AGE Age', 'DM.AGEU Unit', 'DM.SEX Sex',
       'ICF.ICFDAT Informed consent time',
       'EX.EXYN Was the subject vaccinated',
       'EX.EXREASND If no, please specify reason', 'EX.EXREASNO Other Specify',
       'EX.EXNUM Vaccine number', 'EX.EXTIM Vaccination time',
       'PC.PCTIM Time of sample collection',
       'PC.PCPERF Blood sample collection', 'PC.PCVOL Sample volume',
       'RAN.RANYN Was the subject enrolled', 'RAN.RANREA 未随机原因',
       'RAN.RANREAO Other', 'RAN.RANDAT1 Randomization date',
       'RAN.RANNUM Randomization number',
       'IE.IEYN1 Did the subject fultill all inclusion criteria',
       'IE.IECO1 Specify',
       'IE.IEYN2 Did the subject fulfill any exclusion criteria',
       'IE.IECO2 Specify'],
      dtype='object')

In [26]:
data = data.dropna(axis=0, how='all').dropna(axis=1, how='all')

pivoted_data = data.pivot_table(index='Subject Id Screening No.', columns='Visit Visit', values='PC.PCTIM Time of sample collection', aggfunc='first').reset_index()


In [18]:
merged_data = data.groupby('Subject Id Screening No.').first().reset_index()
merged_data = merged_data.merge(pivoted_data, on='Subject Id Screening No.', how='left')

In [30]:
merged_data.drop(['Project No. Protocol No.', 'Visit Visit'], axis =1)

,Subject Id Screening No.,Site No. Site No.,Form Sequence Seq No.,DM.BRTHDAT Date of Birth,DM.AGE Age,DM.AGEU Unit,DM.SEX Sex,ICF.ICFDAT Informed consent time,EX.EXYN Was the subject vaccinated,"EX.EXREASND If no, please specify reason",...,RAN.RANDAT1 Randomization date,RAN.RANNUM Randomization number,IE.IEYN1 Did the subject fultill all inclusion criteria,IE.IECO1 Specify,IE.IEYN2 Did the subject fulfill any exclusion criteria,IE.IECO2 Specify,V2--V1（D0）,V4--V3（D14）,V5--V4（D28）,V6--V5（D90）
0,S-001,1,1,1996-02-06,26.0,years old,M,2022-07-07,1.0,NaN,...,2022-07-11 10:41,A004,1.0,NaN,2.0,None,2022-07-11 09:29,2022-07-25 13:15,2022-08-08 11:26,2022-10-04 09:48
1,S-002,1,1,1980-09-18,41.0,years old,F,2022-07-07,NaN,NaN,...,None,None,1.0,NaN,1.0,3,NaN,NaN,NaN,NaN
2,S-003,1,1,1976-09-28,45.0,years old,F,2022-07-07,1.0,NaN,...,2022-07-11 10:40,A003,1.0,NaN,2.0,None,2022-07-11 10:18,2022-07-25 12:01,2022-08-08 10:49,2022-10-04 13:19
3,S-004,1,1,1966-12-12,55.0,years old,F,2022-07-07,NaN,NaN,...,None,None,1.0,NaN,1.0,14 and 15,NaN,NaN,NaN,NaN
4,S-005,1,1,1995-07-24,26.0,years old,F,2022-07-07,NaN,NaN,...,None,None,1.0,NaN,1.0,14,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,S-489,1,1,1976-06-14,46.0,years old,M,2022-12-12,1.0,NaN,...,2022-12-14 17:19,B405,1.0,NaN,2.0,None,2022-12-14 17:14,2022-12-28 09:30,2023-01-11 10:00,2023-03-18 08:44
489,S-490,1,1,2003-01-16,19.0,years old,M,2022-12-12,1.0,NaN,...,2022-12-14 14:12,B396,1.0,NaN,2.0,None,2022-12-14 14:07,2022-12-28 14:21,2023-01-10 14:50,2023-03-11 11:40
490,S-491,1,1,2001-10-26,21.0,years old,F,2022-12-12,NaN,NaN,...,None,None,1.0,NaN,1.0,10,NaN,NaN,NaN,NaN
491,S-492,1,1,2000-06-28,22.0,years old,M,2022-12-12,1.0,NaN,...,2022-12-14 14:14,B397,1.0,NaN,2.0,None,2022-12-14 14:09,2022-12-28 14:25,2023-01-10 14:48,2023-03-15 11:29


In [23]:
output_file = '/content/output.xlsx'

merged_data.to_excel(output_file, index=False)

# Function to color rows with data
def color_rows_with_data(ws):
    gray_fill = PatternFill(start_color="C0C0C0", end_color="C0C0C0", fill_type="solid")
    white_fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")
    
    for row in ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row):
        has_data = False
        for cell in row:
            if cell.value is not None and cell.column_letter != 'A':
                has_data = True
                break
        if has_data:
            for cell in row:
                if cell.value is None:
                    cell.fill = gray_fill
                else:
                    cell.fill = white_fill

# Apply the function to the output file
wb = load_workbook(output_file)
ws = wb.active
color_rows_with_data(ws)
wb.save(output_file)